In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True) #frozen = true means we are creating immutable data class
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file:Path
    unzip_dir: Path

In [2]:
import os
os.chdir("../")

In [12]:
from RoadClassifier.constants import *
from RoadClassifier.utils.utils import read_yaml, create_directory

In [3]:
%pwd

'c:\\Personal_Projects\\All_Project\\clean-littered-road-classification'

In [13]:
class ConfigurationManager():
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])

In [14]:
ConfigurationManager()

[2024-12-26 17:54:46,588 : INFO utils] : yaml file: configs\config.yaml loaded successfully


[2024-12-26 17:54:46,610 : INFO utils] : yaml file: params.yaml loaded successfully
[2024-12-26 17:54:46,623 : INFO utils] : created directory at: artifacts


In [15]:
import os
print("Current Working Directory:", os.getcwd())
print("Absolute path to config:", Path("configs/config.yaml").absolute())

Current Working Directory: c:\Personal_Projects\All_Project\clean-littered-road-classification
Absolute path to config: c:\Personal_Projects\All_Project\clean-littered-road-classification\configs\config.yaml


In [16]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [17]:
import os
import urllib.request as request
from zipfile import ZipFile

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                self.config.source_url,
                filename = self.config.local_data_file
            )

    def get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg")]

    def preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        

    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self.get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self.preprocess(zf, f, self.config.unzip_dir)

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2024-12-26 17:56:12,684 : INFO utils] : yaml file: configs\config.yaml loaded successfully
[2024-12-26 17:56:12,693 : INFO utils] : yaml file: params.yaml loaded successfully
[2024-12-26 17:56:12,693 : INFO utils] : created directory at: artifacts
[2024-12-26 17:56:12,699 : INFO utils] : created directory at: artifacts/data_ingestion
